In [ ]:
from IPython import get_ipython
get_ipython().magic('reset -sf') 

In [ ]:
from tkinter import filedialog
from tkinter import Tk
import os

root = Tk()
fpath = filedialog.askopenfilename(initialdir = os.getcwd(), title = 'Please select a file:')

root.destroy()

assert fpath[fpath.rindex('/') + 1:].find('.') == -1

In [ ]:
import pickle

with open(fpath,'rb') as f:

    locations_new = pickle.load(f)
    addresses = pickle.load(f)
    distances = pickle.load(f)
    routes = pickle.load(f)
    north_bound = pickle.load(f)
    south_bound = pickle.load(f)
    left_bound = pickle.load(f)
    right_bound = pickle.load(f)
    nc = pickle.load(f)
    nr = pickle.load(f)
    service_time_customer = pickle.load(f)
    service_time_rs = pickle.load(f)
    v = pickle.load(f)
    r = pickle.load(f)
    L = pickle.load(f)
    T = pickle.load(f)

In [ ]:
D = [0]
C = [i for i in range(1, nc + 1)]
R = [i for i in range(nc + 1, nc + nr + 1)]
V = D + C + R
print(V)

labels = ['d'] + ['c' for i in range(0, nc)] + ['r' for i in range(0, nr)]
print(labels)

times_s = {}
for i in C:
    times_s[i] = service_time_customer
for i in R:
    times_s[i] = service_time_rs
    
print(times_s)

In [ ]:
from MCWS import MCWS

mcws = MCWS()

mcws.set_vertices(V, labels, distances, times_s)
mcws.set_parameters(v, r, L, T)

mcws.run()

In [ ]:
print(mcws.get_log())

In [ ]:
tours = mcws.get_tours()

for i in tours:
    if i[1] > i[-2]:
        i.reverse()
        
from operator import itemgetter

tours = sorted(tours, key = itemgetter(1))

print(tours)

In [ ]:
mcws.get_total_distance()

In [ ]:
mcws.get_total_time()

In [ ]:
api_file = open("apikey.txt", "r")
api_key = api_file.read()
api_file.close()

lats, lngs = zip(*locations_new)

import gmplot

gmap = gmplot.GoogleMapPlotter((north_bound + south_bound) / 2, (left_bound + right_bound) / 2, 13, apikey = api_key)

gmap.marker(north_bound, left_bound, color = 'gray')
gmap.marker(south_bound, left_bound, color = 'gray')
gmap.marker(north_bound, right_bound, color = 'gray')
gmap.marker(south_bound, right_bound, color = 'gray')

gmap.marker(lats[0], lngs[0], color = 'red')

gmap.scatter(lats[1: (nc + 1)], lngs[1: (nc + 1)], color = 'blue', size = 25, marker = False)

for i in range(0, nr):
    gmap.marker(lats[1 + nc + i], lngs[1 + nc + i], color = 'gold')

import polyline
for i in range(0, len(tours)):
    for j in range(0, len(tours[i]) - 1):
        pll = routes[(tours[i][j], tours[i][j+1])]['routes'][0]['overview_polyline']['points']
        path = zip(*polyline.decode(pll))
        gmap.plot(*path, edge_width = 3, color = 'green', alpha = 0.5)
        
for i in range(len(lats)):
    gmap.text(lats[i] - 0.0007, lngs[i], str(i))

url = 'map_' + fpath[fpath.rindex('/') + 1:] + '_MCWS.html'

gmap.draw(url)

from IPython.display import IFrame

IFrame(url, width = 500, height = 500)

In [ ]:
from openpyxl import load_workbook

wb = load_workbook('Results.xlsx')
ws = wb.active
ws.append([fpath[fpath.rindex('/') + 1:] + '_MCWS', mcws.get_total_time(), mcws.get_total_distance(), str(tours), str(D), str(C), str(R), v, r, L, T, service_time_customer, service_time_rs])
wb.save('Results.xlsx')